# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) 156034, Wojciech Bogacz <br>
2) 156039, krzysztof Skrobała

In [1]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from IPython.display import display

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [2]:
### Model here:
x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)
x3 = LpVariable(name="x3", lowBound=0)

model = LpProblem(name="max_problem", sense=LpMaximize)

# Constraints
model += (x1 + x2 <= 10, "constraint_1")
model += (2*x2 + x3 <= 12, "constraint_2")
model += (3*x1 + 2*x3 <= 15, "constraint_3")
model += (x1 + x2 + x3 <= 20, "constraint_4")


# Objective function
model += (4*x1 + 2*x2 + 3*x3)
model

max_problem:
MAXIMIZE
4*x1 + 2*x2 + 3*x3 + 0
SUBJECT TO
constraint_1: x1 + x2 <= 10

constraint_2: 2 x2 + x3 <= 12

constraint_3: 3 x1 + 2 x3 <= 15

constraint_4: x1 + x2 + x3 <= 20

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

In [3]:
### Solve here


status = model.solve()
print(status)
# print values of variables
print(f"x1: {x1.varValue}")
print(f"x2: {x2.varValue}")
print(f"x3: {x3.varValue}")

# print optimal value of the objective function
print(f"objective: {model.objective.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/wojtek/Documents/uczelnia/oper_research/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c2b75510027a4e8c915bf3afb01ab3f7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/c2b75510027a4e8c915bf3afb01ab3f7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 22 RHS
At line 27 BOUNDS
At line 28 ENDATA
Problem MODEL has 4 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-1) rows, 3 (0) columns and 6 (-3) elements
0  Obj -0 Dual inf 10.499997 (3)
3  Obj 31.428571
Optimal - objective value 31.428571
After Postsolve, objective 31.428571, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 31.42857143 - 3 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds)

1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

In [2]:
### Initialize the data here:
### TODO
c = np.array([4, 2, 3])
b = np.array([10, 12, 15, 20])
A = np.array([[1, 1, 0], [0, 2, 1], [3, 0, 2], [1, 1, 1]])
M = np.concatenate((A, np.eye(4)), axis=1)


<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [7]:
def getFinalTableau(base, c, b, A, M):
    B_inv = M[:, base]
    c_b = np.concatenate((c, np.zeros(4)),axis =0)
    c_b = c_b[base]
    try:
        B_inv = np.linalg.inv(B_inv)
    except:
        return None
    
    slack_part = np.dot(c_b, B_inv)
    var_part = np.dot(slack_part, A) - c
    return B_inv, c_b, slack_part, var_part

def create_table(tableau):
    top = (tableau[3], tableau[2], tableau[1].dot(tableau[0]).dot(b))
    top = np.concatenate([np.atleast_1d(x) for x in top])
    bottom = np.concatenate([np.concatenate([tableau[0].dot(A),tableau[0]],axis=1),
                                      np.reshape(tableau[0].dot(b), (-1, 1))],axis=1)
    table = np.vstack([top,bottom])
    return table


final_tableaus = []
not_inv = 0
for base in itertools.combinations(range(7), r=4):
    base = np.array(base)
    final_tableau = getFinalTableau(base, c, b, A, M)
    if final_tableau is not None:
        final_tableaus.append(final_tableau)
    else:
        not_inv += 1

licz = 0
feasible = 0 
not_feasible = 0
for tableau in final_tableaus:
    licz += 1

    if np.all(tableau[0].dot(b) >= -0.0001):
        feasible += 1
        if np.all(tableau[3] >= -0.0001):        
          
            table = create_table(tableau)
            df = pd.DataFrame(table)
            df.columns = ['x1', 'x2', 'x3', 's1', 's2', 's3', 's4', 'rs']
            df.index = ['z','1', '2', '3', '4']
            display(df)
        else:
            pass
    else:
        not_feasible += 1

print(f"for {not_inv} bases, matrix B was not invertible")
print(f"There were feasible: {feasible} not feasible: {not_feasible} solutions")



,x1,x2,x3,s1,s2,s3,s4,rs
z,-4.440892e-16,0.0,0.0,0.571429,0.714286,1.142857,0.0,31.428571
1,1.000000e+00,0.0,0.0,0.571429,-0.285714,0.142857,0.0,4.428571
2,5.551115e-17,1.0,0.0,0.428571,0.285714,-0.142857,0.0,5.571429
3,-1.110223e-16,0.0,1.0,-0.857143,0.428571,0.285714,-0.0,0.857143
4,0.000000e+00,0.0,0.0,-0.142857,-0.428571,-0.285714,1.0,9.142857


for 4 bases, matrix B was not invertible
There were feasible: 8 not feasible: 23 solutions


## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

In [6]:
### Model here:
### TODO
### Model here:
y1 = LpVariable(name="y1", lowBound=0)
y2 = LpVariable(name="y2", lowBound=0)
y3 = LpVariable(name="y3", lowBound=0)
y4 = LpVariable(name="y4", lowBound=0)


model = LpProblem(name="dual_problem", sense=LpMinimize)

# Constraints
model += (y1 + 3*y3 + y4 >= 4, "constraint_1")
model += (y1 + 2*y2 + y4 >= 2, "constraint_2")
model += (y2 + 2*y3 + y4>= 3, "constraint_3")


# Objective function
model += (10*y1 + 12*y2 + 15*y3 + 20*y4)
model

dual_problem:
MINIMIZE
10*y1 + 12*y2 + 15*y3 + 20*y4 + 0
SUBJECT TO
constraint_1: y1 + 3 y3 + y4 >= 4

constraint_2: y1 + 2 y2 + y4 >= 2

constraint_3: y2 + 2 y3 + y4 >= 3

VARIABLES
y1 Continuous
y2 Continuous
y3 Continuous
y4 Continuous

In [7]:
### Solve here:
### TODO

status = model.solve()
print(status)
# print values of variables
print(f"y1: {y1.varValue}")
print(f"y2: {y2.varValue}")
print(f"y3: {y3.varValue}")
print(f"y4: {y4.varValue}")


# print optimal value of the objective function
print(f"objective: {model.objective.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/wojtek/Documents/uczelnia/oper_research/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/61eda31316a64eb28eb63dd8c3704017-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/61eda31316a64eb28eb63dd8c3704017-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 22 RHS
At line 26 BOUNDS
At line 27 ENDATA
Problem MODEL has 3 rows, 4 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 4 (0) columns and 9 (0) elements
0  Obj 0 Primal inf 3.833333 (3)
3  Obj 31.428571
Optimal - objective value 31.428571
Optimal objective 31.42857143 - 3 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

1
y1: 0.57142857
y2: 0.71428571
y3: 1.1428571
y4: 0.0
o

2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [8]:
final_tableaus = []
PRIMAL_DUAL_SOLUTIONS = []
for base in itertools.combinations(range(7), r=4):
    base = np.array(base)
    final_tableau = getFinalTableau(base, c, b, A, M)
    if final_tableau is not None:
        final_tableaus.append(final_tableau)
        
        table = create_table(final_tableau)
        temp = []

        temp.append(table[0][7].item())

        temp.extend([0,0,0,0,0,0,0])
        for i,x in enumerate(base):
            temp[1+x] = table[i+1][7].item()

        if np.all(final_tableau[0].dot(b) >= -0.0001):
            temp.append("Y")
        else:
            temp.append("N")
        
        if np.all(final_tableau[3] >= -0.0001) and np.all(final_tableau[2] >= -0.0001):
            temp.append("Y")
        else:
            temp.append("N")
        
        if temp[-2] == "Y" and temp[-1] == "Y":
            temp.append("OPTIMAL")
            temp.append("OPTIMAL")
            temp.append("Y")
            temp.append("Y")
            
        elif temp[-2] == "Y" and temp[-1] == "N":
            temp.append("SUBOPTIMAL")
            temp.append("SUPEROPTIMAL")
            temp.append("N")
            temp.append("Y")
            
        elif temp[-2] == "N" and temp[-1] == "Y":
            temp.append("SUPEROPTIMAL")
            temp.append("SUBOPTIMAL")
            temp.append("Y")
            temp.append("N")
            
        else:
            temp.append("-")
            temp.append("-")
            temp.append("N")
            temp.append("N")
            
        temp.extend(v.item() for v in table[0][3:7])
        temp.extend(v.item() for v in table[0][:3])

        PRIMAL_DUAL_SOLUTIONS.append(temp)
        

    




In [9]:
### TODO 

### DISPLAY STORED PAIRS OF SOLUTIONS
df = pd.DataFrame(PRIMAL_DUAL_SOLUTIONS, columns = ["Z", "x1", "x2", "x3", "slack1", "slack2", "slack3", "slack4", "P_F", "P_O", "P_STATE", "D_STATE", "D_F", "F_O",
                                                  "y1", "y2", "y3", "y4", "sur1", "sur2", "sur3"])
from IPython.display import HTML
# display(df.style.set_properties(**{
#        'width': '230px',
#        'max-width': '230px',
#    }))
df = df.sort_values(by="Z").reset_index(drop=True)
display(HTML(df.to_html()))

,Z,x1,x2,x3,slack1,slack2,slack3,slack4,P_F,P_O,P_STATE,D_STATE,D_F,F_O,y1,y2,y3,y4,sur1,sur2,sur3
0,-4.000000,0.000000,10.000000,-8.000000,0.00,0.000000,31.0,18.000000,N,N,-,-,N,N,-4.000000e+00,3.000000e+00,0.000000e+00,0.000000,-8.000000e+00,0.000000e+00,0.000000e+00
1,0.000000,0.000000,0.000000,0.000000,10.00,12.000000,15.0,20.000000,Y,N,SUBOPTIMAL,SUPEROPTIMAL,N,Y,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-4.000000e+00,-2.000000e+00,-3.000000e+00
2,12.000000,0.000000,6.000000,0.000000,4.00,0.000000,15.0,14.000000,Y,N,SUBOPTIMAL,SUPEROPTIMAL,N,Y,0.000000e+00,1.000000e+00,0.000000e+00,0.000000,-4.000000e+00,0.000000e+00,-2.000000e+00
3,17.500000,10.000000,0.000000,-7.500000,0.00,19.500000,0.0,17.500000,N,N,-,-,N,N,-5.000000e-01,0.000000e+00,1.500000e+00,0.000000,0.000000e+00,-2.500000e+00,0.000000e+00
4,20.000000,5.000000,0.000000,0.000000,5.00,12.000000,0.0,15.000000,Y,N,SUBOPTIMAL,SUPEROPTIMAL,N,Y,0.000000e+00,0.000000e+00,1.333333e+00,0.000000,0.000000e+00,-2.000000e+00,-3.333333e-01
5,20.000000,0.000000,10.000000,0.000000,0.00,-8.000000,15.0,10.000000,N,N,-,-,N,N,2.000000e+00,0.000000e+00,0.000000e+00,0.000000,-2.000000e+00,0.000000e+00,-3.000000e+00
6,22.500000,0.000000,0.000000,7.500000,10.00,4.500000,0.0,12.500000,Y,N,SUBOPTIMAL,SUPEROPTIMAL,N,Y,0.000000e+00,0.000000e+00,1.500000e+00,0.000000,5.000000e-01,-2.000000e+00,0.000000e+00
7,24.000000,-3.000000,0.000000,12.000000,13.00,0.000000,0.0,11.000000,N,N,-,-,N,N,0.000000e+00,3.333333e-01,1.333333e+00,0.000000,0.000000e+00,-1.333333e+00,0.000000e+00
8,27.000000,0.000000,2.250000,7.500000,7.75,0.000000,0.0,10.250000,Y,N,SUBOPTIMAL,SUPEROPTIMAL,N,Y,0.000000e+00,1.000000e+00,1.000000e+00,0.000000,-1.000000e+00,0.000000e+00,0.000000e+00
9,28.000000,4.000000,6.000000,0.000000,0.00,0.000000,3.0,10.000000,Y,N,SUBOPTIMAL,SUPEROPTIMAL,N,Y,4.000000e+00,-1.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,-4.000000e+00
